In [ ]:
###########import packages##########
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn import ensemble
from sklearn import svm
from sklearn import neighbors
from sklearn import tree
from sklearn.linear_model import BayesianRidge
from sklearn.tree import ExtraTreeRegressor
from sklearn.tree import ExtraTreeClassifier
from sklearn import linear_model
from sklearn.metrics import roc_curve, auc
import lightgbm
import catboost
import xgboost
import shap
import seaborn as sns
from catboost import *
from sklearn.manifold import TSNE
%matplotlib
###########loading data##########
fdata=pd.read_csv('database_filled_CD.csv',encoding="gbk")
raw_data=fdata.loc[:,[
                      'Operating Temperature (℃)',#0
                      'Operating Pressure (bar)',#1
                      'Flow Rate (mL min-1)',#2
                      'Active Area (cm2)', #3
                      'Ir wt. %',#4
                      'Ru wt.%',#5
                      'O wt. %',#6
                      'C wt. %',#7
                      'Pure_0/Supported_1',#8
                      'I/C in Anode',#9
                      'Pt wt. %',#10
                      'I/C in Cathode',#11  
                      'Anode Precious Metal Loading (mg cm-2 Ir/Ru/Pt/Pd)',#12
                      'Cathode Precious Metal Loading (mg cm-2 Pt/Pd)',#13
                      'CCM_0/GDE_1',#14    
                      'Membrane Thickness (μm)',#15
                      'EW',#16
                      'Minimum Stability Current Density (A cm-2)',#17
                      'Maximum Stability Current Density (A cm-2)',#18
                      'Fluctuation period (h)',#19
                      'Stability Test Time (h-1)'#34
                        ]]
###########train test splitting##########
raw_param=raw_data.iloc[:,0:35]
print('ready')


In [ ]:
mat_sp=raw_data.corr(method='spearman')#kendall #pearson
mat_kd=raw_data.corr(method='kendall')
mat_ps=raw_data.corr(method='pearson')

In [ ]:
f, ax= plt.subplots(figsize = (14, 10))
sns.set(font_scale=0.75)
sns.heatmap(mat_ps, 
                annot=True, vmax=1,vmin = 0, xticklabels= True, yticklabels= True, square=True, cmap="gist_heat_r")

In [ ]:
CD_CL=fdata.loc[:,['CL_2V_2A']]

In [ ]:
def summarize(list1):
    total=0
    for ele in range(0, len(list1)):
        total = total + list1[ele]
    return total
from sklearn.decomposition import PCA

In [ ]:
pca_3 = PCA(n_components=3)
chem_3_PCA= pca_3.fit_transform(raw_data)     
print(pca_3.explained_variance_ratio_)

pca_2 = PCA(n_components=2)
chem_2_PCA= pca_2.fit_transform(raw_data)     
print(pca_2.explained_variance_ratio_)

print(summarize(pca_3.explained_variance_ratio_))

print(summarize(pca_2.explained_variance_ratio_))

In [ ]:
tsne_3 = TSNE(n_components=3, random_state=0,init='random',learning_rate=500,n_iter=10000,method='exact')
chem_3_TSNE = tsne_3.fit_transform(raw_param)
tsne_2 = TSNE(n_components=2, random_state=0,init='random',learning_rate=500,n_iter=10000,method='exact')
chem_2_TSNE = tsne_2.fit_transform(raw_param)

In [ ]:
from mpl_toolkits.mplot3d import Axes3D
def plot_3d_PCA(dataset,classification_result):
    cl=classification_result.values[:,0]
    colors = ['red', 'blue','green','orange']
    # 根据主成分分析结果绘制散点图
    fig=plt.figure()
    ax=Axes3D(fig)
    for i in range(len(colors)):
        x = dataset[:, 0][cl == i]
        y = dataset[:, 1][cl == i]
        z = dataset[:, 2][cl == i]
        ax.scatter(x,y,z,c=colors[i])
        ax.set_zlabel('Third Principal Component',fontsize='xx-large')
    # 设置图例，0-9用不同颜色表示
    if max(cl)>1:
        plt.legend(np.array([0,1,2,3]), bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.,fontsize='xx-large')
    else:
        plt.legend(np.array([0,1]), bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.,fontsize='xx-large')
    # 设置坐标标签
    plt.xlabel('First Principal Component',fontsize='xx-large')
    plt.ylabel('Second Principal Component',fontsize='xx-large')
    # 设置标题
    plt.title("PCA Scatter Plot 3D",fontsize='xx-large')
    plt.show()
def plot_2d_PCA(dataset,classification_result):
    cl=classification_result.values[:,0]
    colors = ['red', 'blue','green','orange']
    # 根据主成分分析结果绘制散点图
    fig=plt.figure()
    for i in range(len(colors)):
        x = dataset[:, 0][cl == i]
        y = dataset[:, 1][cl == i]
        plt.scatter(x,y,c=colors[i])
    # 设置图例，0-9用不同颜色表示
    if max(cl)>1:
        plt.legend(np.array([0,1,2,3]), bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.,fontsize='xx-large')
    else:
        plt.legend(np.array([0,1]), bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.,fontsize='xx-large')
    # 设置坐标标签
    plt.xlabel('First Principal Component',fontsize='xx-large')
    plt.ylabel('Second Principal Component',fontsize='xx-large')
    # 设置标题
    plt.title("PCA Scatter Plot 2D",fontsize='xx-large')
    # 显示图形
    plt.show()
def plot_3d_TSNE(dataset,classification_result):
    cl=classification_result.values[:,0]
    colors = ['red', 'blue','green','orange']
    # 根据主成分分析结果绘制散点图
    fig=plt.figure()
    ax=Axes3D(fig)
    for i in range(len(colors)):
        x = dataset[:, 0][cl == i]
        y = dataset[:, 1][cl == i]
        z = dataset[:, 2][cl == i]
        ax.scatter(x,y,z,c=colors[i])
        ax.set_zlabel('Third Principal Component',fontsize='xx-large')
    # 设置图例，0-9用不同颜色表示
    if max(cl)>1:
        plt.legend(np.array([0,1,2,3]), bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.,fontsize='xx-large')
    else:
        plt.legend(np.array([0,1]), bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.,fontsize='xx-large')
    # 设置坐标标签
    plt.xlabel('First Principal Component',fontsize='xx-large')
    plt.ylabel('Second Principal Component',fontsize='xx-large')
    # 设置标题
    plt.title("TSNE Scatter Plot 3D",fontsize='xx-large')
    plt.show()
def plot_2d_TSNE(dataset,classification_result):
    cl=classification_result.values[:,0]
    colors = ['red', 'blue','green','orange']
    # 根据主成分分析结果绘制散点图
    fig=plt.figure()
    for i in range(len(colors)):
        x = dataset[:, 0][cl == i]
        y = dataset[:, 1][cl == i]
        plt.scatter(x,y,c=colors[i])
    # 设置图例，0-9用不同颜色表示
    if max(cl)>1:
        plt.legend(np.array([0,1,2,3]), bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.,fontsize='xx-large')
    else:
        plt.legend(np.array([0,1]), bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.,fontsize='xx-large')
    # 设置坐标标签
    plt.xlabel('First Principal Component',fontsize='xx-large')
    plt.ylabel('Second Principal Component',fontsize='xx-large')
    # 设置标题
    plt.title("TSNE Scatter Plot 2D",fontsize='xx-large')
    # 显示图形
    plt.show()

In [ ]:
plot_3d_PCA(chem_3_PCA,CD_CL)

In [ ]:
plot_2d_PCA(chem_2_PCA,CD_CL)

In [ ]:
plot_3d_TSNE(chem_3_TSNE,CD_CL)

In [ ]:
plot_2d_TSNE(chem_2_TSNE,CD_CL)